In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import udf

In [0]:
# read teh parquest type files
df_oct = spark.read.parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/oct/")
df_nov = spark.read.parquet("/Volumes/workspace/ecommerce/ecommerce_data/parquet/nov/")

In [0]:
display(df_oct.limit(5))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-13T06:25:46.000Z,view,1002544,2053013555631882655,electronics.smartphone,apple,460.51,518958788,e7e27c5c-1e78-4812-9f55-cdc658bb40fe
2019-10-13T06:25:46.000Z,view,3700301,2053013565983425517,appliances.environment.vacuum,vitek,120.93,557977070,7afc206c-7259-4beb-b49e-4662267481e9
2019-10-13T06:25:46.000Z,view,49100004,2127425375913902544,null,null,45.05,514456508,9d6837a5-40df-49de-ba3c-5625f2659230
2019-10-13T06:25:46.000Z,view,9200409,2053013552913973497,computers.peripherals.keyboard,defender,12.56,512530774,df2d048d-c1ae-41bf-b5c3-348617f505a8
2019-10-13T06:25:46.000Z,view,1306558,2053013558920217191,computers.notebook,acer,1801.82,523366823,0c7f0449-74d5-4b0c-bb07-ba04cc7a6681


In [0]:
display(df_nov.limit(5))
# combine the two datasets
df_all = df_oct.union(df_nov)
display(df_all.limit(5))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-11-17T08:43:00.000Z,view,2501799,2053013564003713919,appliances.kitchen.oven,elenberg,46.31,563237118,4368d099-6d19-47cb-95f8-1a235bc2d701
2019-11-17T08:43:00.000Z,view,6400335,2053013554121933129,computers.components.cpu,intel,435.28,551129779,4db2c365-ee85-4437-87ad-23d70bfc01a9
2019-11-17T08:43:00.000Z,view,3701538,2053013565983425517,appliances.environment.vacuum,irobot,1878.81,539845715,bf7d95c0-69e1-40ff-817b-7118652f018b
2019-11-17T08:43:00.000Z,view,26400266,2053013563651392361,null,lucente,119.18,572211322,8e6c63f8-7f34-48b3-a0d2-69bbcec41fa3
2019-11-17T08:43:00.000Z,view,1004659,2053013555631882655,electronics.smartphone,samsung,762.18,512965259,2981c9f9-3905-49de-b8db-453d5134969f


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
2019-10-13T06:25:46.000Z,view,1002544,2053013555631882655,electronics.smartphone,apple,460.51,518958788,e7e27c5c-1e78-4812-9f55-cdc658bb40fe
2019-10-13T06:25:46.000Z,view,3700301,2053013565983425517,appliances.environment.vacuum,vitek,120.93,557977070,7afc206c-7259-4beb-b49e-4662267481e9
2019-10-13T06:25:46.000Z,view,49100004,2127425375913902544,null,null,45.05,514456508,9d6837a5-40df-49de-ba3c-5625f2659230
2019-10-13T06:25:46.000Z,view,9200409,2053013552913973497,computers.peripherals.keyboard,defender,12.56,512530774,df2d048d-c1ae-41bf-b5c3-348617f505a8
2019-10-13T06:25:46.000Z,view,1306558,2053013558920217191,computers.notebook,acer,1801.82,523366823,0c7f0449-74d5-4b0c-bb07-ba04cc7a6681


In [0]:
# Top 5 products by revenue
revenue = df_oct.filter(F.col("event_type") == "purchase") \
    .groupBy("product_id") \
    .agg(F.sum("price").alias("revenue")) \
    .orderBy(F.desc("revenue")).limit(5)

display(revenue)

product_id,revenue
1005115,1.2406807350000069E7
1005105,1.0239248679999985E7
1004249,6730112.920000023
1005135,5567806.640000009
1004767,5430723.430000016


In [0]:
# JOINS - check the users retention (users active in both months)
oct_users = df_oct.select("user_id", "event_type").distinct()
nov_users = df_nov.select("user_id", "event_type").distinct()

df_result = oct_users.join(nov_users, "user_id", "inner")
display(df_result.limit(5))

user_id,event_type,event_type
516430318,view,view
515796168,purchase,view
555481419,view,view
515001129,view,cart
514677897,view,view


left‑anti joins Returns ONLY the rows from the LEFT table where NO match exists in the RIGHT table
returns only the left table’s columns.

In [0]:
# left join - find the users that are active in october but not in november

df_result = oct_users.join(nov_users, "user_id", "leftanti")
display(df_result.limit(5))

user_id,event_type
524457561,view
559699559,view
559700544,view
534960222,view
519239709,view


Left-semi - Returns only the rows from the LEFT table that have a match in the RIGHT table.
Does NOT return any columns from the right table.


In [0]:
# left-semi pull matching records from the left table unlike inner it fetches only left table columns.
df_result = oct_users.join(nov_users, "user_id", "leftsemi")
display(df_result.limit(5))

In [0]:
#outer join- find the products that are active in either months
products_oct = df_oct.select("product_id").distinct()
products_nov = df_nov.select("product_id").distinct()

all_products = products_oct.join(products_nov, on="product_id", how="outer")
display(all_products.limit(5))

product_id
1005159
6902812
7004004
8500290
23700185


windows functions

In [0]:
# Running total per user
window = Window.partitionBy("user_id").orderBy("event_time")
df_win= df_oct.withColumn("cumulative_events", F.count("*").over(window))
display(df_win.select("event_time","event_type","user_id","cumulative_events").limit(5))


event_time,event_type,user_id,cumulative_events
2019-10-09T10:30:19.000Z,view,205053188,1
2019-10-09T10:30:44.000Z,view,205053188,2
2019-10-20T18:29:45.000Z,view,209714031,1
2019-10-20T18:30:08.000Z,view,209714031,2
2019-10-24T18:18:25.000Z,view,209714031,3


In [0]:
# event type conversion rate by category
df_result = df_oct.groupBy("category_code") \
    .pivot("event_type") \
    .count() \
    .withColumn(
        "conversion_rate",
        F.col("purchase") / F.col("view") * 100
    )
display(df_result.limit(5))


category_code,cart,purchase,view,conversion_rate
auto.accessories.parktronic,null,46,12305,0.3738317757009346
furniture.living_room.sofa,null,1084,215471,0.5030839416905292
stationery.cartrige,106,134,7380,1.8157181571815717
sport.bicycle,693,838,128759,0.6508282916145668
apparel.sock,7,21,2621,0.8012209080503624


In [0]:
#How long does it take a user to move from view to cart to purchase

w = Window.partitionBy("user_id").orderBy("event_time")

df_journey = df_all.withColumn("prev_event", F.lag("event_type").over(w))\
    .withColumn("prev_time", F.lag("event_time").over(w))\
    .withColumn("time_diff_sec", F.unix_timestamp("event_time") - F.unix_timestamp("prev_time"))
display(df_journey.select("event_time", "event_type", "prev_event", "prev_time", "time_diff_sec").limit(5))

event_time,event_type,prev_event,prev_time,time_diff_sec
2019-11-27T04:33:16.000Z,view,null,null,null
2019-11-27T04:35:24.000Z,view,view,2019-11-27T04:33:16.000Z,128
2019-11-08T07:44:45.000Z,view,null,null,null
2019-11-21T14:11:26.000Z,view,view,2019-11-08T07:44:45.000Z,1146401
2019-11-28T04:58:01.000Z,view,null,null,null


In [0]:
# top selling brand in both month

df_prd_sales = df_all.filter("event_type = 'purchase'") \
    .groupBy("product_id") \
    .agg(F.count("*").alias("purchase_count"))

w = Window.orderBy(F.desc("purchase_count"))

df_ranked = df_prd_sales.withColumn(
    "rank",
    F.rank().over(w)
)
display(df_ranked.limit(5))

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


product_id,purchase_count,rank
1004856,61265,1
1004767,44419,2
1005115,34787,3
4804056,30181,4
1004833,26183,5


In [0]:
# find min and max price for each brand in df_all
price_stats = df_all.filter(F.col("brand").isNotNull())\
    .groupBy("brand")\
    .agg(
        F.min("price").alias("min_price"),
        F.max("price").alias("max_price"))\
    .orderBy(F.asc("brand"))

display(price_stats)

brand,min_price,max_price
a-case,1.21,154.44
a-derma,5.15,22.39
a-elita,8.49,73.28
a-mega,25.74,218.8
aardwolf,8.98,84.43
abk,11.56,11.56
abris,14.44,14.44
absolutechampion,10.3,166.03
abtoys,6.43,115.83
academie,8.72,52.9


In [0]:
# segment products by price

df_bucket = df_all.withColumn(
    "price_bucket",
    F.when(F.col("price") < 220, "Low")
     .when(F.col("price") < 500, "Medium")
     .otherwise("High")
)
display(df_bucket.select("brand","category_code","price","price_bucket") .limit(5))

brand,category_code,price,price_bucket
apple,electronics.smartphone,460.51,Medium
vitek,appliances.environment.vacuum,120.93,Low
null,null,45.05,Low
defender,computers.peripherals.keyboard,12.56,Low
acer,computers.notebook,1801.82,High


In [0]:
#  Category‑specific price buckets

df_clean = df_all.filter(F.col("brand").isNotNull())

w = Window.partitionBy("category_code").orderBy("price")

df_cat_bucket = (
    df_clean
        .withColumn("price_percentile", F.ntile(3).over(w))
        .withColumn(
            "price_bucket",
            F.when(F.col("price_percentile") == 1, "Low")
             .when(F.col("price_percentile") == 2, "Medium")
             .otherwise("High")
        )
)

display(df_cat_bucket.select("brand", "category_code", "price", "price_bucket").limit(5))

brand,category_code,price,price_bucket
muqajankz,accessories.wallet,11.58,Low
muqajankz,accessories.wallet,11.58,Low
muqajankz,accessories.wallet,11.58,Low
muqajankz,accessories.wallet,11.58,Low
muqajankz,accessories.wallet,11.58,Low


In [0]:

from pyspark.sql.types import StringType

def get_main_category(cat):
    if cat is None:
        return None
    return cat.split(".")[0] 

main_cat_udf = udf(get_main_category, StringType())
df_with_main = df_all.withColumn(
    "main_category",
    main_cat_udf("category_code")
)

display(df_with_main.select("category_code", "main_category").limit(10))

category_code,main_category
electronics.smartphone,electronics
appliances.environment.vacuum,appliances
null,null
computers.peripherals.keyboard,computers
computers.notebook,computers
electronics.audio.headphone,electronics
appliances.kitchen.dishwasher,appliances
furniture.living_room.cabinet,furniture
furniture.bedroom.bed,furniture
computers.peripherals.monitor,computers
